# Cryptography

## Recover a secret message from k out of n clues
The idea is that you have a secret message you'd like to store securely, like a BitCoin address. You'd like to store it distributively, using a set of n clues. For example one clue could be written on a piece of paper stored in a safe, one could be on your mobile, one could be stored offline on a USB stick, another could be on a PC. The challenge is to find a way to set up the clues so that secret can be retreived as long as you can retrieve any k out of the n clues.

## Encoding process
- Secret message is integer s
- Choose a prime $p>s$ which acts as a private key and must be retained
- Decide on the number of clues, k, required to recover the secret
- Choose a set of k-1 random integers $r<p$ which must be subsequently destroyed, otherwise secret can be found with one clue, p and r
- Create and store n clues, where each clue is a pair of values (x, Polynomial(x)) for integers x, where $Polynomial(x) = s + \sum r(i) * x^i   mod(p)$

## Decoding process
- To recover s, take any k clues 
- Now you have k points on a curve of degree k, so it is possible to find all the coefficients of the Polynomial
- However, you only care about the secret, s, which is the intercept when x=0
- First create a matrix of the coefficients of the set of k simultaneous equations and recursively eliminate the r values
- This leaves you with a solution for s in the form of a linear congruence equation a*s = b modulo p
- To solve this, first find the inverse of a using the extended Euclid algorithm, then use it to multiply both sides of the equation
- http://www.a-calculator.com/congruence/

# Encoding process

In [1]:
import numpy as np

In [2]:
s = 19950627                     # The secret message
p = 961748941                    # Private key (must be >s)

k = 6                            # Number of clues required to recover secret
n = 2*k                          # Number of clues created
r = np.random.choice(p-1, k-1)+1 # Disposable set of (k-1) random numbers

The clues are simply the coodinates of (any integer) points on the line defined by a polynomial of degree k-1: $Y = s + r_1*x + r_2*x^2 + ... r_{(k-1)}*x^{(k-1)} $ $modulo(p)$. The repeated use of %p was required to avoid overflow.

In [3]:
clues = []
for i in np.random.choice(p-1,n):
    clues += [[i+1] + [sum([s] + [(i+1)**j%p*r[j-1]%p for j in range(1,k)])%p]]
clues

[[781527205, 410540233],
 [385678624, 877135722],
 [127296355, 472483139],
 [296140952, 77522490],
 [242728467, 657244301],
 [907459656, 568109658],
 [797678832, 97606440],
 [876122713, 532957230],
 [126476507, 64798025],
 [151968672, 612585069],
 [756620407, 644163821],
 [421794615, 777360898]]

# Decoding process

Find k random keys

In [4]:
found = [clues[i] for i in np.random.choice(n, k, replace=False)]
found

[[781527205, 410540233],
 [126476507, 64798025],
 [797678832, 97606440],
 [127296355, 472483139],
 [151968672, 612585069],
 [242728467, 657244301]]

Create a a matrix, M, of coeficients $Y = [1, x, x^2, .. x^{(k-1)}] * [s, r_1, r_2, ... r_{(k-1)}].T $. Note that the vector of Y values is included as the first column of M, as this makes the elimination easier in the next step. 

In [5]:
M = np.array([[f[-1],1]+[f[0]**j%p for j in range(1,k)] for f in found])
M

array([[410540233,         1, 781527205, 498058580, 782968403, 392156113,
        187091068],
       [ 64798025,         1, 126476507, 250257611, 780684041, 118466177,
        960190269],
       [ 97606440,         1, 797678832, 770543228, 292201909, 285728078,
        118282168],
       [472483139,         1, 127296355, 275216057, 845295987, 624396016,
        297251133],
       [612585069,         1, 151968672, 910959525, 934579978, 797610222,
        901028813],
       [657244301,         1, 242728467, 216665686,  44053505, 658073828,
        205270568]])

Now eliminate all the r coefficients from the k simultaneous equations, to obtain a single equation for s in the form a*s = b mod p. This is easy when k=2, as we just have 2 points on the line. More generally we can elimiate the variables recursively.

In [6]:
def eliminate(M):
    """Recursive function to eliminate one row and column at a time"""
    N = M.copy()
    rows = len(M)
    if rows == 1:
        return M
    for r in range(rows):
        for s in range(rows):
            if r != s:
                N[r] = (N[r] * M[s,-1])%p
    for r in range(rows-1):
        N[r] = (N[r] - N[-1])%p
    M = N[:-1,:-1]
    return eliminate(M)
    
    

In [7]:
b,a = eliminate(M)[0]  # Note b is the Y value and a is the coefficient of s
a,b

(16228509, 957582198)

Now we need to solve for s where a*s = b mod p <br>
One way to do it with (a, b, p) = (5, 19, 37) is like this, but it is hard to programme
- $5*s=19mod(37)$
- $s = 19/5 = 19*8/40 = 152/3 = 152*13/2 = 26$


Instead, use extended Euclid algorithm. Code borrowed from https://brilliant.org/wiki/extended-euclidean-algorithm/

In [8]:
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcd = b
    return gcd, x, y

egcd(5,37)

(1, 15, -2)

Obtain the solution by multiplying both sides of equation by the inverse of a, which is ouput x of the function

In [9]:
'Secret message obtained from {} clues is {}'.format(k, egcd(a,p)[1]*b%p)

'Secret message obtained from 6 clues is 19950627'